# Learning To use the Bible

## Using Holistic on an image

In [1]:
import os
from natsort import natsorted
#Constants
FRAMES_PATH = "../backend/dynamic_signs/frames"

def __extract_prefix(filename:str, separator:str = "_"):
    return filename.split(separator)[0]

T_filepaths = list[str]
def get_image_sequences_from_dir(dir:str) -> dict[str, list[T_filepaths]]:
    labels = [folder for folder in os.listdir(dir)
                  if os.path.isdir(dir + os.sep + folder)]
    res_dict = {}
    for label in labels:
        folder_path = dir + os.sep + label + os.sep
        files = natsorted([file for file in os.listdir(folder_path)
                               if os.path.isfile(folder_path + file)])

        cur_sequence = []
        label_sequences: list[T_filepaths] = [cur_sequence]
        prev_prefix = __extract_prefix(files[0])
        for image in files:
            prefix = __extract_prefix(image)
            if not (prefix == prev_prefix):
                cur_sequence = []
                label_sequences.append(cur_sequence)
            cur_sequence.append(folder_path + image)
            prev_prefix = prefix
        res_dict[label] = label_sequences
    return res_dict

LABEL_FILES_DICT = get_image_sequences_from_dir(FRAMES_PATH)


In [2]:
from typing import Any, NamedTuple
import mediapipe.python.solutions.holistic as mp_holistic
import cv2 as cv
import csv

mp = mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=1,
)

def get_attributes_as_dict(obj : NamedTuple) -> dict[str, Any]:
    return {field : getattr(obj, field) for field in obj._fields}

def write_processed_sequence_to_csv(label:str, id: int, 
                                    mp_process_results: list[NamedTuple],
                                    verbose = False):
        with open(f"{label}_out.csv", 'a', newline="") as f:        
            writer = csv.writer(f)
            for res in mp_process_results:
                res_as_dict = get_attributes_as_dict(res)
                #Sort to get:
                # face_landmarks, left_hand_landmarks, pose_landmarks, pose_world_landmarks, right_hand_landmarks, segmentation_mask
                for body_part, landmarks in sorted(res_as_dict.items(), key = lambda key_value : key_value[0]):
                    toWrite = []
                    which = body_part.removesuffix("_landmarks")
                    if landmarks is not None: 
                        toWrite = list(sum([ (mrk.x, mrk.y, mrk.z) for mrk in landmarks.landmark], ()))
                    else:
                        if verbose:
                            print(f"No landmarks for {which}")
                    writer.writerow([which, id, *toWrite])
            
            

for label, img_sequences in LABEL_FILES_DICT.items():
    for idx, sequence in enumerate(img_sequences):
        results = [ mp.process(cv.imread(img_path)) for img_path in sequence] 
        print(f"MediaPipe for {label} has {len(results)} many elements")
        write_processed_sequence_to_csv(label, idx, results)


2024-04-04 14:15:31.679292: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 14:15:31.679731: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 14:15:31.682177: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 14:15:31.714134: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 14:15:32.684289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

MediaPipe for TEST has 72 many elements
MediaPipe for NOT_TEST has 50 many elements
MediaPipe for NOT_TEST has 6 many elements


In [3]:
import os
from pathlib import Path
HolisticSequence = dict[str, list[float]]


class csv_reader:
    @staticmethod
    def spawn_sequence() -> HolisticSequence:
        return {"face": [],
                "left_hand": [],
                "pose" : [],
                "right_hand" : [],
                }

    def __init__(self, sequence_spawner = spawn_sequence):
        self.new_holistic_sequence = sequence_spawner

    def _avoid(self, row_val: str):
        return row_val == "segmentation_mask" or row_val == "pose_world"
    
    def _remove_file_suffix(self, file_name: str):
        return file_name.removesuffix("_out.csv")

    def extract_holistic_landmarks(self, path:Path) -> dict[int, HolisticSequence]:
        res :dict[int, HolisticSequence] = {}
        with open(path, 'r') as f:
            reader = csv.reader(f)
            prev_id = -1

            cur_entry: HolisticSequence = self.new_holistic_sequence()
            for row in reader:
                row_key = row[0]
                if self._avoid(row_key):
                    continue

                new_id = int(row[1])
                is_new_video = new_id != prev_id
                if is_new_video and prev_id != -1:
                    res[prev_id] = cur_entry
                    cur_entry = self.new_holistic_sequence()
                    
                prev_id = new_id
                if len(row) > 2:
                    landmarks = row[2:]
                    row_marks = list(map(lambda elm : float(elm), landmarks))
                    if row_key not in cur_entry:
                        raise ValueError(f"Holistic Sequence only allows keys: {[k for k in self.new_holistic_sequence().keys()]}.\n\tEither update \"spawn_sequence\" function or check if csv is broken")
                    cur_entry[row_key].extend( row_marks)
            if len(list(cur_entry.values())[0]) > 0:
                res[prev_id] = cur_entry
        return res
    
    def extract_holistic_landmarks_from_folder(self, path: str) -> dict[str, dict[int, HolisticSequence]]:
        """
            Returns a dictionary from LABEL of the sign to a dictionary of sequence_ID to a HolisticSequence.
                
                HolisticSequence:
                    A dictionary of keys: ["face", "right_hand", "left_hand", "pose"]. 
                    Keys map to a list of the floats corresponding to xyz of landmarks of all frames in the sequnce.
        """
        res = {}
        for csv_file in [path+os.sep+file for file in os.listdir(path) if file.endswith(".csv")]:
            path_to_file = Path(csv_file)
            label = self._remove_file_suffix(path_to_file.name)
            res[label] = self.extract_holistic_landmarks(path_to_file.absolute())
        return res

reader = csv_reader()
result = reader.extract_holistic_landmarks_from_folder("./csvs")
f"Parsed {len(result)} classes"

'Parsed 2 classes'

In [8]:
## BORROW FROM DYNAMIC GESTURE
import numpy as np
from typing import Tuple
from sign.trajectory import TrajectoryBuilder
from sign.landmarks import NormalizedLandmark, pre_process_landmark, calc_landmark_list
from dynamic_signs.csv_reader import csv_reader
bob = TrajectoryBuilder(target_len=24)


def extract_training_data_and_labels_from_dynamic_gesture_map(gesture_map: dict[str, list[np.ndarray]]) -> Tuple[list[np.ndarray], list[str]]:
    trajectories_and_landmarks: list[np.ndarray] = []
    labels : list[str] = []
    for label, label_data in gesture_map.items():
        for data in label_data:
            labels.append(label)
            trajectories_and_landmarks.append(data)

    return trajectories_and_landmarks, labels

def prune_training_data_and_labels_from_dynamic_gesture_csv(input: dict[str, dict[int, list[float]]]) -> dict[str, list[np.ndarray]]:
    target_length = 24*3*21
    bob = TrajectoryBuilder(target_len=target_length)
    res: dict[str, list[np.ndarray]] = {}
    for label, videos in input.items():
        for _, frames in videos.items():
            existing = res.get(label)
            if len(frames) < target_length:
                frames = bob.pad_sequences_of_landmarks(frames)
            else: 
                frames = bob.extract_keyframes_sample(frames)
            frames = np.array(frames)
            if existing is not None:
                existing.append(frames)
            else:
                res[label] = [frames]
                
    return res
#  DICT:
#   { 
#       49: HolisticSequence
# 
#   }
def extract_left_hand_landmarks_from_holistic(dict: dict[str, dict[int, HolisticSequence]]) -> dict[str, dict[int, list[float]]]:
    res = {}
    for label, video_id_to_holistic_seq in dict.items():
        res[label] = {}
        for id, holy_seq in video_id_to_holistic_seq.items():
            res[label][id] = holy_seq["left_hand"]
    return res

def extract_training_data_and_labels_from_dynamic_gesture_csv() -> Tuple[list[np.ndarray], list[str]]:
    unpruned = reader.extract_holistic_landmarks_from_folder("./csvs")
    unpruned = extract_left_hand_landmarks_from_holistic(unpruned)
    pruned = prune_training_data_and_labels_from_dynamic_gesture_csv(unpruned)
    normalized_landmarks = {}
    
    for key, val in pruned.items():
        for seq in val:
            trajectory = bob.make_trajectory(seq.reshape(-1, 21, 3))
            landmarks = []
            for i in range(0, len(seq), 3):
                landmark = NormalizedLandmark()
                landmark.x = seq[i]
                landmark.y = seq[i+1]
                landmark.z = seq[i+2]
                landmarks.append(landmark)
            normalized_landmarks_for_video = pre_process_landmark(calc_landmark_list(landmarks))
            existing = normalized_landmarks.get(key)
            asd = trajectory.to_float_list()
            asd.extend(normalized_landmarks_for_video)
            with_trajectories = np.array(asd)
            
            if existing is None:
                normalized_landmarks[key] = [with_trajectories]
            else:
                existing.append(with_trajectories)
    return extract_training_data_and_labels_from_dynamic_gesture_map(normalized_landmarks)

extracted_data,extracted_labels = extract_training_data_and_labels_from_dynamic_gesture_csv()
list(map(lambda arr : arr.shape ,extracted_data))

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥


[(1077,), (1077,), (1077,)]

In [20]:
#Holistic on one-armed-bandit looking for hands- Holy nation approves of this one 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model_NOTTEST_or_not = make_pipeline(StandardScaler(),
                          SVC(kernel="poly", degree=6, coef0=1))
model_NOTTEST_or_not.fit(extracted_data, (np.array(extracted_labels, dtype=np.str_) == "NOT_TEST"))

if False:
    from joblib import dump
    dump(model_svm, 'dynamic_model.joblib')
model_NOTTEST_or_not

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(coef0=1, degree=6, kernel='poly'))])